## 項目1 使用するPythonのバージョンを知っておく

In [10]:
!python --version

Python 3.8.5


In [11]:
import sys
print(sys.version_info)
print(sys.version)

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


## 項目2 PEP8スタイルガイドに従う

以下のスタイルガイドがある。

* 空白
* 名前付け
* 式と文
* インポート

## 項目3 bytesとstrの違い

In [15]:
a = b'h\x65llo'
print(list(a))
print(a)

[104, 101, 108, 108, 111]
b'hello'


In [18]:
a = 'a\u0300 propos'
print(list(a))
print(a)

['a', '̀', ' ', 'p', 'r', 'o', 'p', 'o', 's']
à propos


Unicodeサンドイッチ（strとbytesの違いで処理に誤りがおきるので変換する）

In [21]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value # str instance

print(repr(to_str(b'foo')))
print(repr(to_str('bar')))

'foo'
'bar'


In [23]:
def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value # bytes instance

print(repr(to_bytes(b'foo')))
print(repr(to_bytes('bar')))

b'foo'
b'bar'


その他の注意

* strとbytesでは同じように扱えない
  * str、bytes同士は+演算子で結合できる、比較、同値チェックができる
    * str、bytes間ではできないので注意